In [3]:
from dotenv import load_dotenv
import os
import nomics
from pprint import pprint as pp
import json
from time import sleep
import pandas as pd
import requests
import io
from collections import OrderedDict
from copy import deepcopy
from tqdm.notebook import tqdm_notebook as progress_bar
from pathlib import Path
from pathvalidate import is_valid_filename

# initialize the environment
load_dotenv()
API_KEY=os.getenv("API_KEY")

nom = nomics.Nomics(API_KEY)

# get the list of currencies
currency_data_df = pd.read_pickle('..\\downloaded_data\\nomics\\crypto_data.pkl')

desired_blockchains = ["ETH", "BNB", "NEO", "TRX", "HT", "MATIC", "FTM", "SOL", "AVAX"]

currency_list = []
for currency_index in range(len(currency_data_df['currency_id'].to_list())):
    currency = currency_data_df.iloc[currency_index]
    if currency['platform_currency'] in desired_blockchains:
        # avoid getting impossible to save file names
        if is_valid_filename(f"{currency['currency_id']}.pkl"):
            currency_list.append(currency['currency_id'])

response = {}

# get list of already downloaded and pickled currencies
already_downloaded_currencies = []
# already_downloaded_currencies = [x.stem for x in Path('..\\downloaded_data\\nomics\\candles').glob('*.pkl')]

# get daily candles for each currency
for currency in progress_bar(currency_list, desc='Downloading candles'):
    if currency not in already_downloaded_currencies:
        response = nom.Candles.get_candles(interval='1d', currency=currency)
        df = pd.json_normalize(response)
        df.fillna(0, inplace=True)
        # add types
        headers = df.columns.to_list()
        for column in headers:
            if column == 'timestamp':
                df[column] = pd.to_datetime(df[column], infer_datetime_format=True)
            else:
                df[column] = df[column].astype(float)
        # pickle the data
        save_path = Path('..\\downloaded_data\\nomics\\candles\\' + currency + '.pkl').resolve()
        df.to_pickle(save_path)

print("\nCompleted downloading candles...")


Completed downloading candles...


In [ ]:
df.head()

In [2]:
import pandas as pd
adf = pd.read_pickle('..\\downloaded_data\\nomics\\crypto_data.pkl')

In [9]:
adf[adf['currency_id'] == 'GIZA']

,currency_id,price,status,name,market_cap,circulating_supply,volume,rank,high,high_timestamp,num_exchanges
4649,GIZA,127.169763,active,GizaDao,0.0,0.0,0.0,9616,0.0,0,1


In [6]:
from dotenv import load_dotenv
import os
import nomics
from pprint import pprint as pp
import json
from time import sleep
import pandas as pd
import requests
import io
from collections import OrderedDict
from copy import deepcopy
from tqdm.notebook import tqdm_notebook as progress_bar
from pathlib import Path

# initialize the environment
load_dotenv()
API_KEY=os.getenv("API_KEY")

nom = nomics.Nomics(API_KEY)

response = nom.Candles.get_candles(interval='1d', currency='GIZA')

In [7]:
pp(response)

[{'close': '131.68802947',
  'high': '279.92064986',
  'low': '131.46222595',
  'open': '131.46222595',
  'timestamp': '2021-12-04T00:00:00Z',
  'transparent_close': '131.68802947',
  'transparent_high': '279.92064986',
  'transparent_low': '131.46222595',
  'transparent_open': '131.46222595',
  'transparent_volume': '731.91',
  'volume': '731.91',
  'volume_transparency': {'A': '731'}},
 {'close': '51.21414790',
  'high': '177.09334590',
  'low': '51.21414790',
  'open': '73.89895467',
  'timestamp': '2021-12-07T00:00:00Z',
  'transparent_close': '51.21414790',
  'transparent_high': '177.09334590',
  'transparent_low': '51.21414790',
  'transparent_open': '73.89895467',
  'transparent_volume': '1783.05',
  'volume': '1783.05',
  'volume_transparency': {'A': '1783'}}]


In [10]:
# altered version to download hourly candles around the transaction times of certain currencies.

from dotenv import load_dotenv
import os
import nomics
from pprint import pprint as pp
import json
from time import sleep
import pandas as pd
import requests
import io
from collections import OrderedDict
from copy import deepcopy
from tqdm.notebook import tqdm_notebook as progress_bar
from pathlib import Path
from pathvalidate import is_valid_filename
from datetime import datetime

# initialize the environment
load_dotenv()
API_KEY=os.getenv("API_KEY")

nom = nomics.Nomics(API_KEY)

# get the df from runs that want to be time matched
df_path = Path(f'../bt_data_runs/cleaned_archive_max_10d_updated.pkl').resolve()
new_df = pd.read_pickle(df_path)
filtered_df = new_df[(new_df['sell_price_multiple']==20) & (new_df['status']=='closed')].reset_index(drop=True)


# get the list of currencies
currency_list = filtered_df['currency_id'].to_list()
currency_list = list(set(currency_list))        # remove duplicates

response = {}

# get list of already downloaded and pickled currencies
already_downloaded_currencies = []
# already_downloaded_currencies = [x.stem for x in Path('..\\downloaded_data\\nomics\\candles').glob('*.pkl')]

# get hourly candles for each currency starting at the transaction time - 6hrs and going for 24 hours
for currency in progress_bar(currency_list, desc='Downloading candles'):
    if currency not in already_downloaded_currencies:
        start_time = filtered_df[filtered_df['currency_id'] == currency]['buy_date'].iloc[0] - pd.Timedelta(hours=6)
        end_time = start_time + pd.Timedelta(hours=24)
        start_time = start_time.tz_localize(tz='UTC')
        end_time = end_time.tz_localize(tz='UTC')
        response = nom.Candles.get_candles(interval='1h', currency=currency, start=start_time.isoformat(), end=end_time.isoformat())
        df = pd.json_normalize(response)
        df.fillna(0, inplace=True)
        # add types
        headers = df.columns.to_list()
        for column in headers:
            if column == 'timestamp':
                df[column] = pd.to_datetime(df[column], infer_datetime_format=True)
            else:
                df[column] = df[column].astype(float)
        # pickle the data
        save_path = Path('..\\downloaded_data\\nomics\\hrly_candles\\' + currency + '.pkl').resolve()
        df.to_pickle(save_path)

print("\nCompleted downloading candles...")


Completed downloading candles...


'2017-06-03T18:00:00+00:00'